In [62]:
import tensorflow as tf
from tensorflow.python.keras.models import Sequential
from tensorflow.python.keras.layers import Input, Dense, GRU, Embedding,LSTM, CuDNNLSTM, Flatten, MaxPooling1D
from tensorflow.python.keras.layers import BatchNormalization, Dropout, TimeDistributed, Conv1D, Conv2D
from tensorflow.keras.optimizers import Adam

import matplotlib.pyplot as plt
import h5py
import pandas as pd
import numpy as np
from time import time

import os
import sys

In [45]:
module_path = os.path.abspath(os.path.join('../../../'))
if module_path not in sys.path:
    sys.path.append(module_path)

In [46]:
from label_transforms import *
from src.utils.train_test_split import train_test_split

In [47]:
path = "../../../files/out/low_level/video_data_low_level_A220.hdf5"
f = h5py.File(name=path, mode='r')
x = np.array(f['x'])
y = np.array(f['y'])

# Running on angry and happy

In [48]:
ang_idx = emotion_abr_to_emotion_id["ang"]
print("ang_idx:", ang_idx)
hap_idx = emotion_abr_to_emotion_id["hap"]
print("hap_idx:", hap_idx)

ang_idx: 12
hap_idx: 33


In [49]:
x_ang_hap = x[np.where((y == ang_idx) | (y == hap_idx))]
y_ang_hap = y[np.where((y == ang_idx) | (y == hap_idx))]

In [50]:
x_ang_hap_au04 = x_ang_hap[:, :, 2]

In [52]:
x_ang_hap_au04.shape

(16, 1084)

In [53]:
y_ang_hap.shape

(16,)

In [58]:
x_ang_au04 = x_ang_hap_au04[np.where(y_ang_hap == ang_idx)]

In [59]:
x_ang_au04.shape

(8, 1084)

In [64]:
plt.plot(x_ang_au04[0])
plt.show()

In [66]:
x_ang_au04 = x_ang_hap_au04[np.where(y_ang_hap == ang_idx)]
x_hap_au04 = x_ang_hap_au04[np.where(y_ang_hap == hap_idx)]

fig, axes = plt.subplots(nrows=1, ncols=2, figsize=(15, 3))
for vid_idx in range(8):
    axes[0].set_title("Anger")
    axes[1].set_title("Happiness")
    ts_ang = x_ang_au04[vid_idx, :500]
    ts_hap = x_hap_au04[vid_idx, :500]
    axes[0].plot(ts_ang)
    axes[1].plot(ts_hap)
    print("hej")

hej
hej
hej
hej
hej
hej
hej
hej


In [19]:
y_ang_hap[y_ang_hap == ang_idx] = 0
y_ang_hap[y_ang_hap == hap_idx] = 1

In [23]:
X_train, X_test, y_train, y_test = train_test_split(x_ang_hap_au04, y_ang_hap, test_size = 0.3)

In [24]:
X_train = X_train.reshape(X_train.shape[0], X_train.shape[1], 1)
X_test = X_test.reshape(X_test.shape[0], X_test.shape[1], 1)

In [25]:
seq_len=1084
STEPS = 50
EPOCHS= 60

NAME = f"td_suDB-{seq_len}seq_{int(time())}"
model_d = Sequential()


# model_d.add(BatchNormalization(input_shape=(seq_len,1)))
model_d.add(LSTM(units=100,
              return_sequences=True,
              input_shape=(seq_len,1)))

# model_d.add(Dropout(0.5))
# model_d.add(TimeDistributed(Dense(100, activation='relu')))

model_d.add(Flatten())
# model_d.add(Dropout(0.5))
model_d.add(Dense(100, activation='relu'))
# model_d.add(Dense(100, activation='relu'))

# model_d.add(Dropout(0.5))
model_d.add(Dense(1, activation='sigmoid'))


model_d.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
model_d.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm (LSTM)                  (None, 1084, 100)         40800     
_________________________________________________________________
flatten (Flatten)            (None, 108400)            0         
_________________________________________________________________
dense (Dense)                (None, 100)               10840100  
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 101       
Total params: 10,881,001
Trainable params: 10,881,001
Non-trainable params: 0
_________________________________________________________________


In [26]:
model_d.fit(X_train, y_train, batch_size=500, epochs=EPOCHS, validation_data=(X_test, y_test))

Epoch 1/60
1/1 [==============================] - 5s 5s/step - loss: 0.6702 - accuracy: 0.4545 - val_loss: 0.1533 - val_accuracy: 0.8000
Epoch 2/60
1/1 [==============================] - 2s 2s/step - loss: 0.4691 - accuracy: 0.4545 - val_loss: 0.1050 - val_accuracy: 1.0000
Epoch 3/60
1/1 [==============================] - 2s 2s/step - loss: 0.3343 - accuracy: 1.0000 - val_loss: 0.0567 - val_accuracy: 1.0000
Epoch 4/60
1/1 [==============================] - 2s 2s/step - loss: 0.1665 - accuracy: 1.0000 - val_loss: 0.0124 - val_accuracy: 1.0000
Epoch 5/60
1/1 [==============================] - 2s 2s/step - loss: 0.0377 - accuracy: 1.0000 - val_loss: 0.0010 - val_accuracy: 1.0000
Epoch 6/60
1/1 [==============================] - 2s 2s/step - loss: 0.0033 - accuracy: 1.0000 - val_loss: 4.3349e-05 - val_accuracy: 1.0000
Epoch 7/60
1/1 [==============================] - 2s 2s/step - loss: 1.4146e-04 - accuracy: 1.0000 - val_loss: 1.0641e-06 - val_accuracy: 1.0000
Epoch 8/60
1/1 [=============

KeyboardInterrupt: 

In [29]:
np.round(model_d.predict(X_test))

array([[0.],
       [1.],
       [0.],
       [0.],
       [0.]], dtype=float32)

In [28]:
y_test

array([0, 1, 0, 0, 0])

# Running on all emotions and action units

In [30]:
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.3)

In [31]:
seq_len=1084
STEPS = 50
EPOCHS= 60

model_a = Sequential()

model_a.add(LSTM(units=100,
              return_sequences=True,
              input_shape=(seq_len,17)))

model_a.add(Flatten())
model_a.add(Dense(100, activation='relu'))

model_a.add(Dense(44, activation='sigmoid'))


model_a.compile(loss=tf.keras.losses.SparseCategoricalCrossentropy(), optimizer='adam', metrics=['accuracy'])
model_a.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_1 (LSTM)                (None, 1084, 100)         47200     
_________________________________________________________________
flatten_1 (Flatten)          (None, 108400)            0         
_________________________________________________________________
dense_2 (Dense)              (None, 100)               10840100  
_________________________________________________________________
dense_3 (Dense)              (None, 44)                4444      
Total params: 10,891,744
Trainable params: 10,891,744
Non-trainable params: 0
_________________________________________________________________


In [32]:
model_a.fit(X_train, y_train, batch_size=50, epochs=EPOCHS, validation_data=(X_test, y_test))

Epoch 1/60
5/5 [==============================] - 13s 2s/step - loss: 4.6753 - accuracy: 0.0488 - val_loss: 4.0383 - val_accuracy: 0.0566
Epoch 2/60
5/5 [==============================] - 10s 2s/step - loss: 3.1738 - accuracy: 0.1667 - val_loss: 3.6971 - val_accuracy: 0.0472
Epoch 3/60
5/5 [==============================] - 10s 2s/step - loss: 2.5350 - accuracy: 0.3455 - val_loss: 3.7321 - val_accuracy: 0.0849
Epoch 4/60
5/5 [==============================] - 10s 2s/step - loss: 1.9202 - accuracy: 0.4919 - val_loss: 3.9009 - val_accuracy: 0.1226
Epoch 5/60
5/5 [==============================] - 10s 2s/step - loss: 1.3697 - accuracy: 0.6545 - val_loss: 4.0952 - val_accuracy: 0.1698
Epoch 6/60
5/5 [==============================] - 10s 2s/step - loss: 0.9458 - accuracy: 0.7520 - val_loss: 4.2746 - val_accuracy: 0.1415
Epoch 7/60
5/5 [==============================] - 10s 2s/step - loss: 0.5516 - accuracy: 0.8943 - val_loss: 4.6957 - val_accuracy: 0.1698
Epoch 8/60
5/5 [==================

KeyboardInterrupt: 

# Smoothing data with moving averages

In [33]:
def moving_average(a, n=3) :
    ret = np.cumsum(a, dtype=float)
    ret[n:] = ret[n:] - ret[:-n]
    return ret[n - 1:] / n

In [36]:
x_ang_hap_au04.shape

(16, 1084)

In [40]:
x_ang_au04 = x_ang_hap_au04[np.where(y_ang_hap == ang_idx)]

In [42]:
x_ang_au04.shape

(0, 1084)

In [65]:
x_ang_au04 = x_ang_hap_au04[np.where(y_ang_hap == ang_idx)]
x_hap_au04 = x_ang_hap_au04[np.where(y_ang_hap == hap_idx)]

fig, axes = plt.subplots(nrows=1, ncols=2, figsize=(15, 3))
for vid_idx in range(8):
    axes[0].set_title("Anger")
    axes[1].set_title("Happiness")
    ts_ang = x_ang_au04[vid_idx, :500]
    ts_hap = x_hap_au04[vid_idx, :500]
    axes[0].plot(ts_ang)
    axes[1].plot(ts_hap)